In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from PIL import Image
import os.path

In [2]:
df = pd.read_csv('/Users/maximilianproll/Dropbox (Aalto)/2_data/01_MAVI_unzipped_preprocessed/MAVI2/2015/rap_2015.csv', sep=',')

column translations:
- vuosi = YEAR
- lohkonro = field parcel
- tunnus = identifier
- kasvikoodi = PLANT CODE
- kasvi = PLANT
- lajikekood = VARIETY CODE
- lajike = VARIETY
- pintaala = Property area
- tays_tuho = full crop loss
- ositt_tuho = partial crop loss

In [3]:
df = df.rename(index=str, columns={
    'vuosi' : 'YEAR',
    'lohkonro' : 'field parcel',
    'tunnus' : 'identifier',
    'kasvikoodi' : 'PLANT CODE',
    'kasvi' : 'PLANT',
    'lajikekood' : 'VARIETY CODE',
    'lajike' : 'VARIETY',
    'pintaala' : 'Property area',
    'tays_tuho' : 'full crop loss',
    'ositt_tuho' : 'partial crop loss'})
print('length:', len(df))

df.head(5)

length: 465595


,rowid,YEAR,field parcel,identifier,PLANT CODE,PLANT,VARIETY CODE,VARIETY,Property area,full crop loss,partial crop loss,xmin,xmax,ymin,ymax
0,1,2015,8340031607-A,108884,1400,Kaura,41,AKSELI,130,NaN,NaN,332992.292,333122.202,6750626.879,6750777.447
1,2,2015,8340033324-A,108884,1400,Kaura,41,AKSELI,211,NaN,NaN,332136.192,332326.200,6749464.459,6749614.102
2,3,2015,8340473258-A,108884,1400,Kaura,41,AKSELI,716,NaN,NaN,332180.685,332627.877,6749577.142,6749797.020
3,4,2015,0460023106-A,84735,1310,Rehuohra,82,TOCADA,46,NaN,NaN,596822.261,596931.075,6880077.350,6880137.385
4,5,2015,0460023510-A,84735,1310,Rehuohra,82,TOCADA,339,NaN,NaN,597438.622,597776.129,6879804.776,6880060.952


In [4]:
df = df.fillna(0)
df.head(5)

,rowid,YEAR,field parcel,identifier,PLANT CODE,PLANT,VARIETY CODE,VARIETY,Property area,full crop loss,partial crop loss,xmin,xmax,ymin,ymax
0,1,2015,8340031607-A,108884,1400,Kaura,41,AKSELI,130,0.0,0.0,332992.292,333122.202,6750626.879,6750777.447
1,2,2015,8340033324-A,108884,1400,Kaura,41,AKSELI,211,0.0,0.0,332136.192,332326.200,6749464.459,6749614.102
2,3,2015,8340473258-A,108884,1400,Kaura,41,AKSELI,716,0.0,0.0,332180.685,332627.877,6749577.142,6749797.020
3,4,2015,0460023106-A,84735,1310,Rehuohra,82,TOCADA,46,0.0,0.0,596822.261,596931.075,6880077.350,6880137.385
4,5,2015,0460023510-A,84735,1310,Rehuohra,82,TOCADA,339,0.0,0.0,597438.622,597776.129,6879804.776,6880060.952


In [5]:
print('remove duplicates')

print(df.shape[0] - len(np.unique(df['field parcel'])), 'duplicate entries' )

fieldparcel = df['field parcel']

df = df[fieldparcel.duplicated() == False]

print(df.shape[0] - len(np.unique(df['field parcel'])), 'duplicate entries' )

remove duplicates
27 duplicate entries
0 duplicate entries


create new column: relative crop loss = crop loss / area

In [6]:
df['full crop loss scaled'] = df['full crop loss'] / df['Property area']
df['partial crop loss scaled'] = df['partial crop loss'] / df['Property area']
df.head(4)

,rowid,YEAR,field parcel,identifier,PLANT CODE,PLANT,VARIETY CODE,VARIETY,Property area,full crop loss,partial crop loss,xmin,xmax,ymin,ymax,full crop loss scaled,partial crop loss scaled
0,1,2015,8340031607-A,108884,1400,Kaura,41,AKSELI,130,0.0,0.0,332992.292,333122.202,6750626.879,6750777.447,0.0,0.0
1,2,2015,8340033324-A,108884,1400,Kaura,41,AKSELI,211,0.0,0.0,332136.192,332326.200,6749464.459,6749614.102,0.0,0.0
2,3,2015,8340473258-A,108884,1400,Kaura,41,AKSELI,716,0.0,0.0,332180.685,332627.877,6749577.142,6749797.020,0.0,0.0
3,4,2015,0460023106-A,84735,1310,Rehuohra,82,TOCADA,46,0.0,0.0,596822.261,596931.075,6880077.350,6880137.385,0.0,0.0


In [7]:
i_full = df[ (df['full crop loss scaled'] == 1) & (df['partial crop loss scaled'] == 0)].index
df.loc[i_full, 'category'] = 'full'

i_partial = df[ (df['partial crop loss scaled'] == 1) & (df['full crop loss scaled'] <= 0)].index
df.loc[i_partial, 'category'] = 'partial'

i_noloss = df[ (df['full crop loss scaled'] == 0) & (df['partial crop loss scaled'] == 0)].index
df.loc[i_noloss, 'category'] = 'noloss'

i_fullandpartial = df[
    (df['full crop loss scaled'] > 0) & (df['full crop loss scaled'] < 1) & 
    (df['partial crop loss scaled'] > 0) & (df['partial crop loss scaled'] < 1)
].index
df.loc[i_fullandpartial, 'category'] =  'fullandpartial'

threshold = 0.5
i_anyloss = df[
    (df['full crop loss scaled'] > threshold) | (df['partial crop loss scaled'] > threshold)
].index
df.loc[i_anyloss, 'category_2'] =  'anyloss'

df.loc[[
    i_full[0],
    i_partial[0],
    i_noloss[0],
    i_fullandpartial[0],
    i_anyloss[0]
]]

,rowid,YEAR,field parcel,identifier,PLANT CODE,PLANT,VARIETY CODE,VARIETY,Property area,full crop loss,partial crop loss,xmin,xmax,ymin,ymax,full crop loss scaled,partial crop loss scaled,category,category_2
545,546,2015,0050503250-A,84256,1310,Rehuohra,AN,BRAGE,30,30.0,0.0,342108.034,342185.802,6992174.301,6992254.342,1.000000,0.000000,full,anyloss
318,319,2015,0050122627-A,76,1400,Kaura,39,RINGSAKER,158,0.0,158.0,347229.931,347370.234,6990306.741,6990440.868,0.000000,1.000000,partial,anyloss
0,1,2015,8340031607-A,108884,1400,Kaura,41,AKSELI,130,0.0,0.0,332992.292,333122.202,6750626.879,6750777.447,0.000000,0.000000,noloss,NaN
528,529,2015,9260281418-A,54012,1400,Kaura,20,FIIA,485,200.0,285.0,412690.216,413191.332,7143419.884,7143935.577,0.412371,0.587629,fullandpartial,anyloss
318,319,2015,0050122627-A,76,1400,Kaura,39,RINGSAKER,158,0.0,158.0,347229.931,347370.234,6990306.741,6990440.868,0.000000,1.000000,partial,anyloss


In [8]:
print('variety codes: ' , len(np.unique(list(df['VARIETY CODE']))))
print('varieties: ' , len(np.unique(list(df['VARIETY']))))
print('plant codes:', len(np.unique(list(df['PLANT CODE']))))
print('plants:', len(np.unique(list(df['PLANT']))))

variety codes:  155
varieties:  553
plant codes: 58
plants: 58


In [9]:
np.unique(list(df['PLANT']))

array(['Apila', 'Auringonkukka', 'Hirssi', 'Humala', 'Härkäpapu', 'Kaura',
       'Kevätrapsi', 'Kevätruis', 'Kevätrypsi', 'Kevätspelttivehnä',
       'Kevätvehnä', 'Kuituhamppu', 'Kuitunokkonen', 'Kuitupellava',
       'Kvinoa (kinua)', 'Mailanen', 'Maissi', 'Makealupiini',
       'Mallasohra', 'Mesikkä', 'Muut valkuaiskasvit', 'Muut viljat',
       'Rehuherne', 'Rehuohra', 'Ruistankio (Camelina, Kitupellava)',
       'Ruokaherne', 'Ruokaperuna', 'Ruokateollisuusperuna',
       'Seos (herne/härkäpapu/makea lupiini/öljykasvit)',
       'Seos herne/härkäpapu/makea lupiini yli 50 %+viljaa',
       'Seoskasvusto (valkuaiskasvit)',
       'Seoskasvusto (valkuaiskasvit+vilja)',
       'Seoskasvusto (valkuaiskasvit+öljykasvit)',
       'Seoskasvusto (vilja+öljykasvit)', 'Seoskasvusto (viljat)',
       'Seoskasvusto (öljykasvit)',
       'Siemenperuna (sertifioidun siemenen tuotantoon)', 'Soijapapu',
       'Sokerimaissi', 'Syysohra', 'Syysrapsi', 'Syysruis',
       'Syysruisvehnä', 'Syysryps

translation of plants:

- Kaura - Oats
- Kevät rapsi - Spring rape
- Kevät ruis - Spring rye
- Kevät rypsi - Spring rapeseed
- Kevätvehnä - spring wheat
- Mallasohra - Malting barley
- Rehuherne - feed peas
- Rehu ohra - feed barley
- Ruoka herne - Food Pea
- Ruoka peruna - Food potato
- Ruoka teollisuus peruna - Food industry potato
- Seoskasvusto (valkuaiskasvit+vilja) - Seagrass (protein crops + grain)
- Seoskasvusto (viljat) - Seagrass (cereals)
- Siemen peruna (sertifioidunsiemenentuotantoon) - Potato seed (for certified seed production)
- Syys ruis - September rye
- Syys rypsi - Autumn is rapeseed
- Syysvehnä - Winter wheat
- Tärkkelysperuna - Starch potatoes
- Tärkkelys perunanomasiemenlisäys - Starch Potato Meal Addition
- Vihantavilja(kaura) - Forage cereals (oats)
- Vihantavilja(ohra) - Forage cereals (barley)
- Öljypellava - Flax oil

In [10]:
plants = []
numbers = []
fulls, partials, nolosses, fullandpartials = [], [], [], []
anylosses = []

for plant in np.unique(df['PLANT']):
    plants.append(plant)
    df_tmp = df[df['PLANT'] == plant]
    numbers.append(len(df_tmp))
    fulls.append(len(df_tmp[df_tmp['category'] == 'full']))
    partials.append(len(df_tmp[df_tmp['category'] == 'partial']))
    nolosses.append(len(df_tmp[df_tmp['category'] == 'noloss']))
    fullandpartials.append(len(df_tmp[df_tmp['category'] == 'fullandpartial']))
    anylosses.append(len(df_tmp[df_tmp['category_2'] == 'anyloss']))
#     print(plant, '\t ', num_tmp)

In [11]:
df_numbers = pd.DataFrame(
    {'plant': plants,
     'number': numbers,
     'full': fulls, 
     'partial': partials, 
     'noloss': nolosses, 
     'fullandpartial': fullandpartials,
     'anyloss': anylosses
    })
df_numbers = df_numbers.sort_values(by=['anyloss'], ascending=False)

In [12]:
n = 5 
df_numbers.head(n+2)

,plant,number,full,partial,noloss,fullandpartial,anyloss
23,Rehuohra,158304,1960,3516,150437,1761,7310
5,Kaura,130707,419,2083,127418,500,3055
18,Mallasohra,22334,190,545,21097,337,1119
10,Kevätvehnä,60400,46,826,59301,121,1004
8,Kevätrypsi,11102,147,230,10604,88,468
26,Ruokaperuna,12604,36,198,12326,27,266
47,Tärkkelysperuna,2602,13,181,2374,20,221


top 5
~ 6k noloss + full + partial + fullandpartial+ anyloss

6k choice by area: largest first


In [13]:
top_5_plants = df_numbers.sort_values(by=['anyloss'], ascending=False).plant[0:n].values

drop Rehuohra because it is already downloaded

In [14]:
top_5_plants = np.delete(top_5_plants, np.where(top_5_plants=='Rehuohra'))
print(top_5_plants)

['Kaura' 'Mallasohra' 'Kevätvehnä' 'Kevätrypsi']


reduce df to selected plants

In [15]:
df = df[(df.PLANT == top_5_plants[0]) | (df.PLANT == top_5_plants[1]) | (df.PLANT == top_5_plants[2]) | (df.PLANT == top_5_plants[3])]
df.sample(10)

,rowid,YEAR,field parcel,identifier,PLANT CODE,PLANT,VARIETY CODE,VARIETY,Property area,full crop loss,partial crop loss,xmin,xmax,ymin,ymax,full crop loss scaled,partial crop loss scaled,category,category_2
430396,430397,2015,2330121771-A,87454,1400,Kaura,33,PEPPI,24,0.0,0.0,301868.405,301931.266,7001772.552,7001848.540,0.0,0.0,noloss,NaN
318082,318083,2015,9750251656-A,68471,1400,Kaura,17,BELINDA,309,0.0,0.0,276397.395,276629.418,6985236.944,6985579.563,0.0,0.0,noloss,NaN
13287,13288,2015,5890217690-A,8190,1120,Kevätvehnä,15,KRUUNU,214,0.0,0.0,294894.178,295102.946,6946987.217,6947161.521,0.0,0.0,noloss,NaN
190912,190913,2015,4260675385-A,5866,1400,Kaura,41,AKSELI,140,0.0,0.0,626861.269,627018.871,6943638.888,6943810.603,0.0,0.0,noloss,NaN
303352,303353,2015,6170064411-A,92751,1400,Kaura,41,AKSELI,63,0.0,0.0,438884.160,438953.864,7120470.753,7120670.265,0.0,0.0,noloss,NaN
332923,332924,2015,9360490094-A,96228,1400,Kaura,41,AKSELI,678,0.0,0.0,329048.888,329287.826,6923030.814,6923559.029,0.0,0.0,noloss,NaN
4057,4058,2015,2330454706-A,101664,1400,Kaura,41,AKSELI,92,0.0,0.0,312781.536,312897.899,7007614.919,7007772.571,0.0,0.0,noloss,NaN
433582,433583,2015,0860417076-A,57398,1320,Mallasohra,AG,HARBINGER,69,0.0,0.0,387475.503,387555.054,6737749.691,6737855.762,0.0,0.0,noloss,NaN
261106,261107,2015,5451010279-A,63656,1400,Kaura,06,LEILA,425,0.0,0.0,216123.849,216324.256,6952559.300,6952858.347,0.0,0.0,noloss,NaN
305030,305031,2015,6810524893-A,79274,1400,Kaura,39,RINGSAKER,111,0.0,0.0,557871.869,558047.138,6873879.731,6874002.386,0.0,0.0,noloss,NaN


reduce df to only those fields, which are either full, partial, fullandpartial or anyloss and select the largest 6k fields with no loss

In [16]:
i_losses, i_noloss6ks = [], []
for plant in top_5_plants:
    df_plant = df[df.PLANT == plant]
    i_losses += df_plant[(df_plant.category == 'full') | (df_plant.category == 'partial') | (
                df_plant.category == 'fullandpartial') | (df_plant.category_2 == 'anyloss')].index.values.tolist()
    i_noloss6ks += df_plant[
                       (df_plant.category == 'noloss')
                   ].sort_values(by=['Property area'], ascending=False).index[0:6000].values.tolist()

df = df[(df.index.isin(i_losses)) | df.index.isin(i_noloss6ks)]

In [17]:
# check that all the remaining indeces have to be either in i_losses or in i_nolosses6ks
np.any(np.array([df.index.isin(i_losses), df.index.isin(i_noloss6ks)]),axis=0).all()

True